## iGEM labeling functions

## example

In [65]:
import pandas as pd
import numpy as np
import os
import nltk
import regex as re
import ast
from collections import defaultdict
# from snorkel.labeling import labeling_function

In [7]:
# @labeling_function()
def oxidation_with_sub_prod(sentence):
    for word in oxidationWords:
        if word in sentence.lower:
            return TRUE
    return ABSTAIN

### test data written by hand

In [8]:
test_data = [
    [["carbon", "oxygen"], "carbon was oxidized by the oxygen"],
    [["carbon", "oxygen", "amino acid", "cassie"], "the cassie amino acid is connected to the carbon which was oxidixed by the oxygen inside the dna"],
    
    [["L-GalDH"], "The active fractions from the gel filtration column were further separated by native PAGE, followed by staining for L-GalDH activity."],
    [["L-GalDH"], "Southern blot analysis revealed that the spinach L-GalDH gene occurs in a single copy."],
    [["L-GalL dehydrogenase"], "We reported that transcripts of L-GalL dehydrogenase (L-GalLDH), which functions at the last step of the biosynthetic pathway, were expressed in leaves, stems and roots at almost the same level (Yabuta et al. 2000)."],
    [["L-GalDH", "dehydro-AsA", "L-GalL"], "The L-GalDH activity was not affected by the addition of 1 mM dehydro-AsA (DHA) or 1 mM L-GalL."]
]

### setting up abstain

In [9]:
ABSTAIN = -1

### labeling functions + small tests written

In [85]:
# LF_general_chemical
# If one of the chemicals is in a list of common chemical terms such as amino acid, sugar, adenosine, amide, adenine, etc..., we label FALSE
# https://www.hach.com/chemGlossary   
common_chemical_terms = ["amino acid", "sugar", "adenosine", "amide", "adenine", "acid", "algaecide", "amines", "base", "biocides",
    "buffer", "clarifier", "diluent", "hydrocarbon", "hydrogenation", "molecule", "nutrients", "polymer", "peptide", "polypeptide", "solute",
    "solvent", "suspended", "mixture", "saturated", "unsaturated"]
# @labeling_function()
def general_chemical(x):
    for chem in x[1]:
        if (chem in common_chemical_terms):
            return (False)
    return ABSTAIN

In [86]:
# LF_adjacent_mentions
# If the chemicals are adjacent, we label FALSE
# @labeling_function()
def adjacent_mentions(x):
    for index in range(len(x[1])):
        for index_2 in range(len(x[1])):
            if ((x[1][index] + " " + x[1][index_2]) in x[0]):
                return False
    return ABSTAIN

In [87]:
# LF_gene_words
# If one of the chemicals is in a list of genetic terms such as reductase, dna, gene, we label FALSE
# 
common_genetic_terms = ["reductase", "dna", "gene", "allele", "locus", "genotype", "phenotype", "dominant", "recessive", "additive", "phenoset",
    "diallelic", "multiallelic", "polyallelic", "monomorphic", "monoallelic", "polymorphism", "mutation", "complex", "trait", "multifactorial",
    "polygenic", "monogenic", "mixed model", "transmission probability", "transition probability", "epistasis", "interaction", "pleiotropy",
    "quantitative trait locus", "probit", "logit", "penetrance", "transformation", "scale of measurement", "identity by descent", "identity in state",
    "haplotype", "phase", "multilocus", "genotype", "allelic", "association", "linkage", "disequilibrium", "gametic"]
# @labeling_function()
def gene_words(x):
    for chem in x[1]:
        if (chem in common_genetic_terms):
            return False
    return ABSTAIN
    

In [88]:
# LF_sep_or
# If the chemicals are separated by or, we label FALSE
# @labeling_function()
def sep_or(x):
    for index in range(len(x[1])):
        for index_2 in range(len(x[1])):
            if (x[1][index] + " or " + x[1][index_2] in x[0]):
                return False
    return ABSTAIN

In [89]:
# LF_sep_and
# If the chemicals are separated by and, we label FALSE
# @labeling_function()
def sep_and(x):
    for index in range(len(x[1])):
        for index_2 in range(len(x[1])):
            if (x[1][index] + " and " + x[1][index_2] in x[0]):
                return False
    return ABSTAIN

In [90]:
# LF_sep_comma
# If the chemicals are separated by a comma, we label FALSE
# @labeling_function()
def sep_comma(x):
    for index in range(len(x[1])):
        for index_2 in range(len(x[1])):
            if (x[1][index] + ", " + x[1][index_2] in x[0]):
                return False
    return ABSTAIN

In [91]:
# LF_sep_via
# If the chemicals are separated by via, in, etc, we label FALSE
common_via_words = ["via", "in"]
# @labeling_function()
def sep_via(x):
    for index in range(len(x[1])):
        for index_2 in range(len(x[1])):
            for term in common_via_words:
                if (x[1][index] + " " + term + " " + x[1][index_2] in x[0]):
                    return False
    return ABSTAIN

In [92]:
# LF_sep_sym
# If the chemicals are separated by a single character, we label FALSE  
# @labeling_function()
def sep_sym(x):
    for index in range(len(x[1])):
        chem_1_index = x[0].find(x[1][index])
        chem_1_len = len(x[1][index])
        while (0 <= chem_1_index < len(x[0])):
            for index_2 in range(len(x[1])):
                chem_2_index = x[0].find(x[1][index_2])
                while (0 <= chem_2_index < len(x[0])):
                    if (sep_sym_helper(chem_1_index, chem_1_len, chem_2_index)):
                        return False
                    chem_2_index = x[0].find(x[1][index_2], chem_2_index + 1)
            chem_1_index = x[0].find(x[1][index], chem_1_index + 1)
    return ABSTAIN

def sep_sym_helper(index_1, length, index_2):
    if (index_1 + length + 1== index_2):
        return True
    return False

In [94]:
# testing separated by a single character (ensuring it works with repeated chemicals)
sep_sym(["cassie is here and her name cassie cas is cas cassie", ["cassie", "cas"]])

False

In [95]:
# LF_followed_ase
# If one of the chemicals is followed by a word that ends with -ase, we label FALSE
# @labeling_function()
def followed_ase(x):
    sentence = x[0].replace('.', '')
    sentence = sentence.replace(',', '')
    words = sentence.split(" ")
    for chem in x[1]:
        index = -1
        while (index < len(words)):
            if not(chem in words[index + 1:]):
                index = len(words)
                break
            if (" " in chem):
                chem_split = chem.split(" ")
                index = words.index(chem_split[-1], index + 1)
            else:
                index = words.index(chem, index + 1)
            if (0 < index + 1 < len(words) and words[index + 1][-3:] == "ase"):
                return False
    return ABSTAIN

In [96]:
# test followed_ase (making sure it works if the chem is repeated)
followed_ase(["carbon fiber and carbon lactase with oxygen", ["carbon", "oxygen"]])

False

In [97]:
# LF_group
# If there is a close mention of a functional chemical group, we label FALSE (as it is more likely descriptive of a structure than of a reaction)
# https://www.masterorganicchemistry.com/2010/10/06/functional-groups-organic-chemistry/
common_functional_groups = ["alkane", "alkene", "alkyne", "benzene ring", "phenyl", "amine", "alcohol", "ether", "alkyl halide", "thiol",
    "aldehyde", "ketone", "ester", "carboxylic acid", "amide", "nitrile", "epoxide", "disulfide", "imine", "acid chloride", "anhydride", "nitro",
    "sulfide", "thioether", "group"]
# @labeling_function()
def group(x):
    for group in common_functional_groups:
        if (group in x[0]):
            return False
    return ABSTAIN


In [98]:
# LF_followed_by_noun
# If one of the chemicals is followed by a noun, we label FALSE
# @labeling_function()
def followed_by_noun(x):
    sentence = x[0].replace(',', '')
    words = sentence.split(" ")
    tagged = nltk.pos_tag(words)
    for chem in x[1]:
        index = -1
        while (index < len(words)):
            if not(chem in words[index + 1:]):
                index = len(words)
                break
            if (" " in chem):
                chem_split = chem.split(" ")
                index = words.index(chem_split[-1], index + 1)
            else:
                index = words.index(chem, index + 1)
            if (index + 1 < len(words)):
                if (tagged[index + 1][1] == "NN"):
                    return False
    return ABSTAIN

In [99]:
# testing followed_by_noun (ensuring it works with repeats)
followed_by_noun(["cassie eating apple, eating for cassie table", ["cassie", "apple"]])

False

In [100]:
# test cases
print(followed_by_noun(["The active fractions from the gel filtration column were further separated by native PAGE, followed by staining for L-GalDH activity.", ["L-GalDH"]]))
print(followed_by_noun(["Southern blot analysis revealed that the spinach L-GalDH gene occurs in a single copy.", ["L-GalDH"]]))
print(followed_by_noun(["We reported that transcripts of L-GalL dehydrogenase (L-GalLDH), which functions at the last step of the biosynthetic pathway, were expressed in leaves, stems and roots at almost the same level (Yabuta et al. 2000).", ["L-GalL dehydrogenase"]]))
print(followed_by_noun(["The L-GalDH activity was not affected by the addition of 1 mM dehydro-AsA (DHA) or 1 mM L-GalL.", ["L-GalDH", "dehydro-AsA", "L-GalL"]]))

False
False
-1
False


In [101]:
# LF_sep_verb
# If the chemicals are separated by a verb, we label TRUE
# @labeling_function()
def sep_verb(x):
    sentence = x[0].replace(',', '')
    words = sentence.split(" ")
    indexes = []
    for chem in x[1]:
        index = -1
        while (index < len(words)):
            if (" " in chem):
                chem_split = chem.split(" ")
                if not(chem_split[-1] in words[index + 1:]):
                    index = len(words)
                    break
                index = words.index(chem_split[-1], index + 1)
            else:
                if not(chem in words[index + 1:]):
                    index = len(words)
                    break
                index = words.index(chem, index + 1)
            indexes.append(index)
    min_index = min(indexes)
    max_index = max(indexes)
    tagged = nltk.pos_tag(words)
    for index in range(min_index + 1, max_index):
        if (tagged[index][1][:2] == "VB"):
            return True
    return ABSTAIN

In [102]:
# testing sep_verb (works when there are several instances)
sep_verb(["cassie cas cassie hi there oxidized cas", ["cassie", "cas"]])

True

In [103]:
# test cases
print(sep_verb(["The active fractions from the gel filtration column were further separated by native PAGE, followed by staining for L-GalDH activity.", ["L-GalDH"]]))
print(sep_verb(["Southern blot analysis revealed that the spinach L-GalDH gene occurs in a single copy.", ["L-GalDH"]]))
print(sep_verb(["We reported that transcripts of L-GalL dehydrogenase (L-GalLDH), which functions at the last step of the biosynthetic pathway, were expressed in leaves, stems and roots at almost the same level (Yabuta et al. 2000).", ["L-GalL dehydrogenase"]]))
print(sep_verb(["The L-GalDH activity was not affected by the addition of 1 mM dehydro-AsA (DHA) or 1 mM L-GalL.", ["L-GalDH", "dehydro-AsA", "L-GalL"]]))

-1
-1
-1
True


In [104]:
# LF_sep_adverb
# If the chemicals are separated by a adverb, we label FALSE
# @labeling_function()
def sep_adverb(x):
    sentence = x[0].replace(',', '')
    words = sentence.split(" ")
    indexes = []
    for chem in x[1]:
        index = -1
        while (index < len(words)):
            if (" " in chem):
                chem_split = chem.split(" ")
                if not(chem_split[-1] in words[index + 1:]):
                    index = len(words)
                    break
                index = words.index(chem_split[-1], index + 1)
            else:
                if not(chem in words[index + 1:]):
                    index = len(words)
                    break
                index = words.index(chem, index + 1)
            indexes.append(index)
    min_index = min(indexes)
    max_index = max(indexes)
    tagged = nltk.pos_tag(words)
    for index in range(min_index + 1, max_index):
        if (tagged[index][1][:2] == "RB"):
            return False
    return ABSTAIN

In [105]:
# testing sep_adverb (for repeated chemicals)
sep_adverb(["cassie happily danced to the moon with cas", ["cassie", "cas"]])

False

In [106]:
# test cases
print(sep_adverb(["The active fractions from the gel filtration column were further separated by native PAGE, followed by staining for L-GalDH activity.", ["L-GalDH"]]))
print(sep_adverb(["Southern blot analysis revealed that the spinach L-GalDH gene occurs in a single copy.", ["L-GalDH"]]))
print(sep_adverb(["We reported that transcripts of L-GalL dehydrogenase (L-GalLDH), which functions at the last step of the biosynthetic pathway, were expressed in leaves, stems and roots at almost the same level (Yabuta et al. 2000).", ["L-GalL dehydrogenase"]]))
print(sep_adverb(["The L-GalDH activity was not affected by the addition of 1 mM dehydro-AsA (DHA) or 1 mM L-GalL.", ["L-GalDH", "dehydro-AsA", "L-GalL"]]))

-1
-1
-1
False


In [108]:
# LF_includes_reaction_words
# If the sentence contains reactions words, we label True
reaction_words = ["convert", "yeild", "produce", "make", "react", "create", "synthesize", "conversion",
    "transformation", "transform", "synthesise", "burn", "rust", "ferment", "explode", "agent", "catalyze"
    "combust", "corrode", "decompose", "oxidize", "neutralize", "oxidization", "neutralization", "combustion",
    "redox", "methylaion", "gas-forming", "displacement", "combination", "exchange", "precipitate",
    "precipitation", "transfer", "through", "by", "produce"]
# @labeling_function()
def includes_reaction_words(x):
    for word in reaction_words:
        if(word in x[0]):
            return True
    return ABSTAIN

In [109]:
# LF_includes_reaction_component_words
# If the sentence contains reaction compontents words, we label True
reaction_component_words = ["substrate", "product", "reactant"]
# @labeling_function()
def includes_reaction_component_words(x):
    for word in reaction_component_words:
        if(word in x[0]):
            return True
    return ABSTAIN

In [110]:
# LF_sep_conversion_words
# If the sentence contains to, from, into, etc., we label True
conversion_words = ["to", "from", "into", "becomes", "became"]
# @labeling_function()
def sep_converstion_words(x):
    sentence = x[0].replace(',', '')
    words = sentence.split(" ")
    indexes = []
    for chem in x[1]:
        index = -1
        while (index < len(words)):
            if (" " in chem):
                chem_split = chem.split(" ")
                if not(chem_split[-1] in words[index + 1:]):
                    index = len(words)
                    break
                index = words.index(chem_split[-1], index + 1)
            else:
                if not(chem in words[index + 1:]):
                    index = len(words)
                    break
                index = words.index(chem, index + 1)
            indexes.append(index)
    min_index = min(indexes)
    max_index = max(indexes)
    for index in range(min_index + 1, max_index):
        if (words[index] in conversion_words):
            return True
    return ABSTAIN

In [111]:
# test sep_conversion_words
sep_converstion_words(["cassie truns into cas", ["cassie", "cas"]])

True

### my todo + notes

In [34]:
# things to fix
    # create specific reation verb labeling (complete)
        # oxidation
        # reduction
        # combustion
        # composition
        # decomposition
        # etc
    # creating more test cases
    # current test sentences from
        # jacob's csv
        # https://academic.oup.com/pcp/article/45/9/1271/1857717
    # ways to clean data
        # ensure there are at least two chemicals (complete)
        # get rid of the greek alphabet (complete)
        # get rid of periods and () and ; (complete)
        # get rid of repeat chems (complete)
    # need to figure out how to deal with things in the sentence twice (complete)

### testing on data written in this file

In [112]:
# changed data format so must change written data format before running
# for data in test_data:
#     print(data)
#     print("general chemicals")
#     print(general_chemical(data[0]))
#     print("adjacent mentions")
#     print(adjacent_mentions(data[0], data[1]))
#     print("sep or")
#     print(sep_or(data[0], data[1]))
#     print("sep and")
#     print(sep_and(data[0], data[1]))
#     print("sep comma")
#     print(sep_comma(data[0], data[1]))
#     print("sep via")
#     print(sep_via(data[0], data[1]))
#     print("sep sym")
#     print(sep_sym(data[0], data[1]))
#     print("followed ase")
#     print(followed_ase(data[0], data[1]))
#     print("group")
#     print(group(data[1]))
#     print("followed by noun")
#     print(followed_by_noun(data[0], data[1]))
#     print("sep verb")
#     print(sep_verb(data[0], data[1]))
#     print("sep adverb")
#     print(sep_adverb(data[0], data[1]))
#     print()

### importing data in and cleaning it

In [70]:
sentence_df = pd.read_csv("sentence_data_cleaned_jacob.csv")
sentence_with_chem_df = sentence_df.dropna()
several_chem_df = sentence_with_chem_df.loc[sentence_with_chem_df["chemicals"].str.contains(",")]
several_chem_df

,doc,sentence_index,sentence,entity_type,label,chemicals,genes,substrates,products,enzymes,label index dict,abstract_clean,abstract_expand,abstract_ordered
197,12067524,0,BACKGROUND AND AIMS: Glutamic acid decarboxyla...,"null, null, null, null, null, null, null, null...","O, O, O, O, B-enzyme, I-enzyme, O, O, B-enzyme...","glutamate, GABA",GAD,glutamate,"gamma-aminobutyric, acid, GABA","Glutamic, acid, GAD, EC, 4.1.1.15","{4: 'B-enzyme', 5: 'I-enzyme', 8: 'B-enzyme', ...",BACKGROUND AND AIMS: Glutamic acid decarboxyla...,['These included a specific enzyme activity of...,['BACKGROUND AND AIMS: Glutamic acid decarboxy...
252,12513997,3,"Interestingly, the allele of PRO1 was shown to...","null, null, null, null, null, GENE-Y, null, nu...","O, O, O, O, O, O, O, O, O, O, O, O, O, B-enzym...","gamma-glutamyl, gamma-glutamyl, L-proline, L-g...",PRO1,L-glutamate,L-proline,"gamma-glutamyl, kinase, gamma-glutamyl, phosphate","{13: 'B-enzyme', 14: 'I-enzyme', 16: 'B-enzyme...",We previously isolated a mutant which showed a...,['The approach described in this paper could b...,['We previously isolated a mutant which showed...
282,12668769,8,We concluded that FDH has no direct role in th...,"null, null, null, GENE-Y, null, null, null, nu...","O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O...","Ser, formate","FDH, FDH",formate,CO,FDH,"{23: 'B-SUBSTRATE', 25: 'B-PRODUCT-OF', 31: 'B...",Serine (Ser) biosynthesis in C(3) plants can o...,"['In shoots, therefore, the pathway from forma...",['Serine (Ser) biosynthesis in C(3) plants can...
318,12742526,6,Even though the activities of MAT and GNMT wer...,"null, null, null, null, null, GENE-N, null, GE...","O, O, O, O, O, B-enzyme, O, B-enzyme, O, O, O,...","S-adenosylmethionine, S-adenosylhomocysteine","MAT, GNMT",S-adenosylmethionine,S-adenosylhomocysteine,"MAT, GNMT","{5: 'B-enzyme', 7: 'B-enzyme', 15: 'B-SUBSTRAT...",Ames dwarf mice (df/df) are deficient in growt...,"['Taken together, the data suggest that methio...",['Ames dwarf mice (df/df) are deficient in gro...
323,12850267,1,One of the enzymes responsible for the product...,"null, null, null, null, null, null, null, null...","O, O, O, O, O, O, O, O, O, B-PRODUCT-OF, O, B-...","KA, kynurenine, glutamine","KATI, GTK",glutamine,"KA, oxoglutaramic, acid","kynurenine, aminotransferase, KATI, GTK, EC, 2...","{9: 'B-PRODUCT-OF', 11: 'B-enzyme', 12: 'I-enz...",Kynurenic acid (KA) is an endogenous glutamate...,['Kynurenic acid (KA) is an endogenous glutama...,['Kynurenic acid (KA) is an endogenous glutama...
439,15132128,0,PURPOSE: The fluoropyrimidine carbamate (capec...,"null, null, null, null, null, null, CHEMICAL, ...","O, O, O, B-SUBSTRATE, I-SUBSTRATE, O, B-SUBSTR...","capecitabine, 5-fluorouracil, 5-FU, thymidine",TP,"fluoropyrimidine, carbamate, capecitabine","5-fluorouracil, 5-FU","thymidine, phosphorylase, TP","{3: 'B-SUBSTRATE', 4: 'I-SUBSTRATE', 6: 'B-SUB...",PURPOSE: The fluoropyrimidine carbamate (capec...,['Favorable enzyme profiles (high TP and low D...,['PURPOSE: The fluoropyrimidine carbamate (cap...
465,15155769,1,Carnitine acetyltransferases (CrAT) catalyze t...,"CHEMICAL, null, null, GENE-Y, null, null, null...","B-enzyme, I-enzyme, O, B-enzyme, O, O, O, O, O...","Carnitine, acetyl-CoA, carnitine, acetylcarnitine",CrAT,"acetyl-CoA, carnitine",acetylcarnitine,"Carnitine, acetyltransferases, CrAT","{0: 'B-enzyme', 1: 'I-enzyme', 3: 'B-enzyme', ...","In eukaryotes, L-carnitine is involved in ener...","['In eukaryotes, L-carnitine is involved in en...","['In eukaryotes, L-carnitine is involved in en..."
484,15689518,0,"L-serine dehydratase (SDH), a member of the be...","CHEMICAL, null, null, GENE-Y, null, null, null...","B-enzyme, I-enzyme, O, B-enzyme, O, O, O, O, O...","L-serine, L-serine, L-threonine, pyruvate, 2-o...",SDH,"L-serine, L-threonine","pyruvate, 2-oxobutyrate","L-serine, dehydratase, SDH","{0: 'B-enzyme', 1: 'I-enzyme', 3: 'B-enzyme', ...","L-serine dehydratase (SDH), a member of the be...","['Furthermore, the activ

In [71]:
sentence_chem_df = several_chem_df[["sentence", "chemicals"]]

In [72]:
greek_alphabet = {
    u'\u0393': 'Gamma',
    u'\u0394': 'Delta',
    u'\u0398': 'Theta',
    u'\u039B': 'Lamda',
    u'\u039E': 'Xi',
    u'\u03A0': 'Pi',
    u'\u03A3': 'Sigma',
    u'\u03A6': 'Phi',
    u'\u03A7': 'Chi',
    u'\u03A8': 'Psi',
    u'\u03A9': 'Omega',
    u'\u03B1': 'alpha',
    u'\u03B2': 'beta',
    u'\u03B3': 'gamma',
    u'\u03B4': 'delta',
    u'\u03B5': 'epsilon',
    u'\u03B6': 'zeta',
    u'\u03B7': 'eta',
    u'\u03B8': 'theta',
    u'\u03B9': 'iota',
    u'\u03BA': 'kappa',
    u'\u03BB': 'lamda',
    u'\u03BC': 'mu',
    u'\u03BD': 'nu',
    u'\u03BE': 'xi',
    u'\u03BF': 'omicron',
    u'\u03C0': 'pi',
    u'\u03C1': 'rho',
    u'\u03C3': 'sigma',
    u'\u03C4': 'tau',
    u'\u03C5': 'upsilon',
    u'\u03C6': 'phi',
    u'\u03C7': 'chi',
    u'\u03C8': 'psi',
    u'\u03C9': 'omega'
}

In [73]:
'β' in greek_alphabet.keys()

True

In [74]:
def remove_greek(words):
    for letter in greek_alphabet.keys():
        words = words.replace(letter, greek_alphabet[letter])
    return words


In [75]:
# greek test
remove_greek("11β-hydroxysteroid")

'11beta-hydroxysteroid'

In [76]:
def chem_into_array(chemicals):
    chemicals = remove_greek(chemicals)
    chemicals = chemicals.lower()
    sample_chems = chemicals.replace("%20", " ")
    sample_chems_list = sample_chems.split(", ")
    for index in range(len(sample_chems_list)):
        while(sample_chems_list[index][-1] == ","):
            sample_chems_list[index] = sample_chems_list[index][:-1]
    sample_chems_list = list(set(sample_chems_list))
    return sample_chems_list

In [77]:
# removing excess commas from the end
chem_into_array("cassie,, cassie, cas")

['cas', 'cassie']

In [78]:
def sentence_cleaned(sentence):
    sentence = remove_greek(sentence)
    sentence = sentence.lower()
    sentence = sentence.replace(".", "")
    sentence = sentence.replace(")", "")
    sentence = sentence.replace("(", "")
    sentence = sentence.replace(";", "")
    sentence = sentence.replace(":", "")
    return sentence

In [79]:
sentence_chem_df["sentence"] = sentence_chem_df["sentence"].apply(sentence_cleaned)
sentence_chem_df

C:\Users\cassi\AppData\Local\Temp/ipykernel_24428/942711163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentence_chem_df["sentence"] = sentence_chem_df["sentence"].apply(sentence_cleaned)


,sentence,chemicals
197,background and aims glutamic acid decarboxylas...,"glutamate, GABA"
252,"interestingly, the allele of pro1 was shown to...","gamma-glutamyl, gamma-glutamyl, L-proline, L-g..."
282,we concluded that fdh has no direct role in th...,"Ser, formate"
318,even though the activities of mat and gnmt wer...,"S-adenosylmethionine, S-adenosylhomocysteine"
323,one of the enzymes responsible for the product...,"KA, kynurenine, glutamine"
439,purpose the fluoropyrimidine carbamate capecit...,"capecitabine, 5-fluorouracil, 5-FU, thymidine"
465,carnitine acetyltransferases crat catalyze the...,"Carnitine, acetyl-CoA, carnitine, acetylcarnitine"
484,"l-serine dehydratase sdh, a member of the beta...","L-serine, L-serine, L-threonine, pyruvate, 2-o..."
545,spermidine/spermine n1-acetyltransferase ssat ...,"polyamine, spermidine, spermine"
561,astrocytes may play a role in these manifestat...,"glutamate, glutamine, glutamine"


In [80]:
sentence_chem_df["chemicals"] = sentence_chem_df["chemicals"].apply(chem_into_array)
sentence_chem_df = sentence_chem_df.loc[sentence_chem_df["chemicals"].str.len() > 1]
sentence_chem_df

C:\Users\cassi\AppData\Local\Temp/ipykernel_24428/2060890707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentence_chem_df["chemicals"] = sentence_chem_df["chemicals"].apply(chem_into_array)


,sentence,chemicals
197,background and aims glutamic acid decarboxylas...,"[gaba, glutamate]"
252,"interestingly, the allele of pro1 was shown to...","[l-proline, l-glutamate, gamma-glutamyl]"
282,we concluded that fdh has no direct role in th...,"[ser, formate]"
318,even though the activities of mat and gnmt wer...,"[s-adenosylhomocysteine, s-adenosylmethionine]"
323,one of the enzymes responsible for the product...,"[glutamine, ka, kynurenine]"
439,purpose the fluoropyrimidine carbamate capecit...,"[capecitabine, thymidine, 5-fu, 5-fluorouracil]"
465,carnitine acetyltransferases crat catalyze the...,"[carnitine, acetylcarnitine, acetyl-coa]"
484,"l-serine dehydratase sdh, a member of the beta...","[pyruvate, l-serine, 2-oxobutyrate, l-threonine]"
545,spermidine/spermine n1-acetyltransferase ssat ...,"[spermidine, spermine, polyamine]"
561,astrocytes may play a role in these manifestat...,"[glutamate, glutamine]"


In [83]:
sentence_chem_df["arguments"] = sentence_chem_df[["sentence", "chemicals"]].values.tolist()
sentence_chem_df

C:\Users\cassi\AppData\Local\Temp/ipykernel_24428/4164902504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentence_chem_df["arguments"] = sentence_chem_df[["sentence", "chemicals"]].values.tolist()


,sentence,chemicals,arguments
197,background and aims glutamic acid decarboxylas...,"[gaba, glutamate]",[background and aims glutamic acid decarboxyla...
252,"interestingly, the allele of pro1 was shown to...","[l-proline, l-glutamate, gamma-glutamyl]","[interestingly, the allele of pro1 was shown t..."
282,we concluded that fdh has no direct role in th...,"[ser, formate]",[we concluded that fdh has no direct role in t...
318,even though the activities of mat and gnmt wer...,"[s-adenosylhomocysteine, s-adenosylmethionine]",[even though the activities of mat and gnmt we...
323,one of the enzymes responsible for the product...,"[glutamine, ka, kynurenine]",[one of the enzymes responsible for the produc...
439,purpose the fluoropyrimidine carbamate capecit...,"[capecitabine, thymidine, 5-fu, 5-fluorouracil]",[purpose the fluoropyrimidine carbamate capeci...
465,carnitine acetyltransferases crat catalyze the...,"[carnitine, acetylcarnitine, acetyl-coa]",[carnitine acetyltransferases crat catalyze th...
484,"l-serine dehydratase sdh, a member of the beta...","[pyruvate, l-serine, 2-oxobutyrate, l-threonine]","[l-serine dehydratase sdh, a member of the bet..."
545,spermidine/spermine n1-acetyltransferase ssat ...,"[spermidine, spermine, polyamine]",[spermidine/spermine n1-acetyltransferase ssat...
561,astrocytes may play a role in these manifestat...,"[glutamate, glutamine]",[astrocytes may play a role in these manifesta...


### running test on the imported data

In [113]:
for index in range(len(sentence_chem_df["sentence"])):
    arguments = sentence_chem_df["arguments"].iloc[index]
    print(arguments)
    print("general chemicals: " + str(general_chemical(arguments)))
    print("adjacent mentions: " + str(adjacent_mentions(arguments)))
    print("sep or: " + str(sep_or(arguments)))
    print("sep and: " + str(sep_and(arguments)))
    print("sep comma: " + str(sep_comma(arguments)))
    print("sep via: " + str(sep_via(arguments)))
    print("sep sym: " + str(sep_sym(arguments)))
    print("followed ase: " + str(followed_ase(arguments)))
    print("group: " + str(group(arguments)))
    print("followed by noun: " + str(followed_by_noun(arguments)))
    print("sep verb: " + str(sep_verb(arguments)))
    print("sep adverb: " + str(sep_adverb(arguments)))
    print("includes reaction words: " + str(includes_reaction_words(arguments)))
    print("sep conversion words: " + str(sep_converstion_words(arguments)))
    print()

['background and aims glutamic acid decarboxylase gad, ec 41115 catalyses the conversion of glutamate to gamma-aminobutyric acid gaba', ['gaba', 'glutamate']]
general chemicals: -1
adjacent mentions: -1
sep or: -1
sep and: -1
sep comma: -1
sep via: -1
sep sym: -1
followed ase: -1
group: -1
followed by noun: -1
sep verb: -1
sep adverb: -1
includes reaction words: True
sep conversion words: True

['interestingly, the allele of pro1 was shown to enhance the activities of gamma-glutamyl kinase and gamma-glutamyl phosphate reductase, both of which catalyze the first two steps of l-proline synthesis from l-glutamate and which together may form a complex in vivo', ['l-proline', 'l-glutamate', 'gamma-glutamyl']]
general chemicals: -1
adjacent mentions: -1
sep or: -1
sep and: -1
sep comma: -1
sep via: -1
sep sym: -1
followed ase: False
group: False
followed by noun: False
sep verb: True
sep adverb: -1
includes reaction words: -1
sep conversion words: True

['we concluded that fdh has no direct 

### new csv uncleaned data

In [63]:
uncleaned_df = pd.read_csv("../sentence_annotations_elsevier_pmid_split6.csv")
uncleaned_df

,lit_id,indices,start,end,sentence,sentence_pos,enzymes,enzyme_locations,chemical_entities_full,chemical_names,chemical_smiles,name_smile_tuples
0,10.1002/jps.20686,0,0,1297,serial JL 313843 291210 291727 291789 291928 3...,"[('serial', 'JJ'), ('JL', 'NN'), ('313843', 'C...",NaN,[],[],NaN,NaN,[]
1,10.1002/jps.20686,1,1298,1324,Published by Elsevier Inc.,"[('Published', 'VBN'), ('by', 'IN'), ('Elsevie...",NaN,[],[],NaN,NaN,[]
2,10.1002/jps.20686,2,1325,1345,All rights reserved.,"[('All', 'DT'), ('rights', 'NNS'), ('reserved'...",NaN,[],[],NaN,NaN,[]
3,10.1002/jps.20686,3,1346,9469,KINETICANALYSESFORSPECIESDIFFERENCESINPGLYCOPR...,[('KINETICANALYSESFORSPECIESDIFFERENCESINPGLYC...,NaN,[],"[{'text': 'Diltiazem', 'start': 1718, 'end': 1...","Diltiazem,, Cyclosporin%20A,, Dexamethasone",COc1ccc(cc1)[C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)[C@...,"[('Diltiazem', 'COc1ccc(cc1)[C@@H]2Sc3ccccc3N(..."
4,10.1002/jps.20686,4,9470,9599,Immunoblot analyses of P-gp expressed in MDR1 ...,"[('Immunoblot', 'NN'), ('analyses', 'NNS'), ('...",NaN,[],"[{'text': 'H241', 'start': 9594, 'end': 9598, ...",H241,C[C@]12C[C@](O)(C[C@@H]1CC[C@@H]3[C@@H]2CC[C@@...,"[('H241', 'C[C@]12C[C@](O)(C[C@@H]1CC[C@@H]3[C..."
...,...,...,...,...,...,...,...,...,...,...,...,...
952542,10.1263/jbb.99.623,154,29565,29683,A novel ATP regeneration system using polyphos...,"[('A', 'DT'), ('novel', 'JJ'), ('ATP', 'NN'), ...","phosphotransferase, kinase","[('phosphotransferase', 9, 9), ('kinase', 12, ...","[{'text': 'ATP', 'start': 29573, 'end': 29576,...",ATP,Nc1ncnc2n(cnc12)C3OC(CO[P](O)(=O)O[P](O)(=O)O[...,"[('ATP', 'Nc1ncnc2n(cnc12)C3OC(CO[P](O)(=O)O[P..."
952543,10.1263/jbb.99.623,155,29684,29723,91 2001 557 563 20 Fujio T. Maruyama A.,"[('91', 'CD'), ('2001', 'CD'), ('557', 'CD'), ...",NaN,[],[],NaN,NaN,[]
952544,10.1263/jbb.99.623,156,29724,29731,Mori H.,"[('Mori', 'NNP'), ('H', 'NNP'), ('.', '.')]",NaN,[],"[{'text': 'H', 'start': 29729, 'end': 29730, '...",H,[H],"[('H', '[H]')]"
952545,10.1263/jbb.99.623,157,29732,29882,Production of useful substances by the couplin...,"[('Production', 'NN'), ('of', 'IN'), ('useful'...",NaN,[],"[{'text': 'ATP', 'start': 29797, 'end': 29800,...","ATP,, ATP",Nc1ncnc2n(cnc12)C3OC(CO[P](O)(=O)O[P](O)(=O)O[...,"[('ATP', 'Nc1ncnc2n(cnc12)C3OC(CO[P](O)(=O)O[P..."


In [49]:
uncleaned_no_na_df = uncleaned_df.dropna()
uncleaned_several_chem_df = uncleaned_no_na_df.loc[uncleaned_no_na_df["chemical_names"].str.contains(",")]
uncleaned_several_chem_df = uncleaned_several_chem_df[["sentence", "chemical_names"]]

In [50]:
uncleaned_several_chem_df["sentence"] = uncleaned_several_chem_df["sentence"].apply(sentence_cleaned)
uncleaned_several_chem_df["chemical_names"] = uncleaned_several_chem_df["chemical_names"].apply(chem_into_array)

In [51]:
uncleaned_several_chem_df["chemical_names"]

150       [dexamethasone, diltiazem, calcein-am, cyclosp...
271       [sodium dodecyl sulfate, geranyl pyrophosphate...
280                                                    [h+]
281                                                    [h+]
286                                                    [h+]
                                ...                        
952414                    [d-alanine-d-alanine, vancomycin]
952450                         [methanol, tetrahydrofurane]
952517                                      [kato, sucrose]
952526                     [d-alanyl-d-alanine, vancomycin]
952541                                [amp, atp, adenylate]
Name: chemical_names, Length: 50212, dtype: object

In [52]:
re_check_several_chem_uncleaned_df = uncleaned_several_chem_df.loc[uncleaned_several_chem_df["chemical_names"].str.len() > 1]

In [53]:
re_check_several_chem_uncleaned_df

,sentence,chemical_names
150,"we selected diltiazem, cyclosporin a, and dexa...","[dexamethasone, diltiazem, calcein-am, cyclosp..."
271,"croteau r washington state univ, inst biol che...","[sodium dodecyl sulfate, geranyl pyrophosphate..."
295,chemicalmodificationschickenliverpyruvatecarbo...,"[lysine, pyruvate, cysteine]"
296,"ash de temple univ, hlth sci ctr, sch med, dep...","[pyruvate, n-(7-dimethylamino-4-methyl-3-couma..."
298,at a one- to two-fold molar excess over active...,"[pyruvate, oxaloacetate, adp]"
...,...,...
952414,key words d-alanine-d-alanine ligase d-amino a...,"[d-alanine-d-alanine, vancomycin]"
952450,the mobile phase consisted of a linear gradien...,"[methanol, tetrahydrofurane]"
952517,weber v falkenhagen d subpol a novel sucrose-b...,"[kato, sucrose]"
952526,active-site mutants of the vanc2 d-alanyl-d-se...,"[d-alanyl-d-alanine, vancomycin]"


In [114]:
re_check_several_chem_uncleaned_df["arguments"] = re_check_several_chem_uncleaned_df[["sentence", "chemical_names"]].values.tolist()
re_check_several_chem_uncleaned_df

C:\Users\cassi\AppData\Local\Temp/ipykernel_24428/1771679872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_check_several_chem_uncleaned_df["arguments"] = re_check_several_chem_uncleaned_df[["sentence", "chemical_names"]].values.tolist()


,sentence,chemical_names,arguments
150,"we selected diltiazem, cyclosporin a, and dexa...","[dexamethasone, diltiazem, calcein-am, cyclosp...","[we selected diltiazem, cyclosporin a, and dex..."
271,"croteau r washington state univ, inst biol che...","[sodium dodecyl sulfate, geranyl pyrophosphate...","[croteau r washington state univ, inst biol ch..."
295,chemicalmodificationschickenliverpyruvatecarbo...,"[lysine, pyruvate, cysteine]",[chemicalmodificationschickenliverpyruvatecarb...
296,"ash de temple univ, hlth sci ctr, sch med, dep...","[pyruvate, n-(7-dimethylamino-4-methyl-3-couma...","[ash de temple univ, hlth sci ctr, sch med, de..."
298,at a one- to two-fold molar excess over active...,"[pyruvate, oxaloacetate, adp]",[at a one- to two-fold molar excess over activ...
...,...,...,...
952414,key words d-alanine-d-alanine ligase d-amino a...,"[d-alanine-d-alanine, vancomycin]",[key words d-alanine-d-alanine ligase d-amino ...
952450,the mobile phase consisted of a linear gradien...,"[methanol, tetrahydrofurane]",[the mobile phase consisted of a linear gradie...
952517,weber v falkenhagen d subpol a novel sucrose-b...,"[kato, sucrose]",[weber v falkenhagen d subpol a novel sucrose-...
952526,active-site mutants of the vanc2 d-alanyl-d-se...,"[d-alanyl-d-alanine, vancomycin]",[active-site mutants of the vanc2 d-alanyl-d-s...


In [115]:
for index in range(0, 20):
    sentence = re_check_several_chem_uncleaned_df["arguments"].iloc[index]
    print(arguments)
    print("general chemicals: " + str(general_chemical(arguments)))
    print("adjacent mentions: " + str(adjacent_mentions(arguments)))
    print("sep or: " + str(sep_or(arguments)))
    print("sep and: " + str(sep_and(arguments)))
    print("sep comma: " + str(sep_comma(arguments)))
    print("sep via: " + str(sep_via(arguments)))
    print("sep sym: " + str(sep_sym(arguments)))
    print("followed ase: " + str(followed_ase(arguments)))
    print("group: " + str(group(arguments)))
    print("followed by noun: " + str(followed_by_noun(arguments)))
    print("sep verb: " + str(sep_verb(arguments)))
    print("sep adverb: " + str(sep_adverb(arguments)))
    print("includes reaction words: " + str(includes_reaction_words(arguments)))
    print("includes reaction component words: " + str(includes_reaction_component_words(arguments)))
    print("sep conversion words: " + str(sep_converstion_words(arguments)))
    print()

['6-dhsg was metabolised by gsh to form a gsh conjugate gs-6-dhsg in raw 2647 cells, via a potential mechanism involving the catalytic activity of glutathione-s-transferase gst', ['gs-6-dhsg', '6-dhsg', 'gsh']]
general chemicals: -1
adjacent mentions: -1
sep or: -1
sep and: -1
sep comma: -1
sep via: -1
sep sym: -1
followed ase: -1
group: -1
followed by noun: False
sep verb: True
sep adverb: -1
includes reaction words: True
includes reaction component words: -1
sep conversion words: True

['6-dhsg was metabolised by gsh to form a gsh conjugate gs-6-dhsg in raw 2647 cells, via a potential mechanism involving the catalytic activity of glutathione-s-transferase gst', ['gs-6-dhsg', '6-dhsg', 'gsh']]
general chemicals: -1
adjacent mentions: -1
sep or: -1
sep and: -1
sep comma: -1
sep via: -1
sep sym: -1
followed ase: -1
group: -1
followed by noun: False
sep verb: True
sep adverb: -1
includes reaction words: True
includes reaction component words: -1
sep conversion words: True

['6-dhsg was m

### snorkel code for model

In [ ]:
# from snorkel.labeling.model import LabelModel
# from snorkel.labeling import PandasLFApplier

In [66]:
# Define the set of labeling functions (LFs)
lfs = [general_chemical, adjacent_mentions, sep_or, sep_and, sep_comma, sep_via, sep_sym, followed_ase, group,
    followed_by_noun, sep_verb, sep_adverb,includes_reaction_words, includes_reaction_component_words,
    sep_converstion_words]

# Apply the LFs to the unlabeled training data
# applier = PandasLFApplier(lfs)
# L_train = applier.apply(df_train)

In [ ]:
# Train the label model and compute the training labels
# label_model = LabelModel(cardinality=2, verbose=True)
# label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
# df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")